In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# Load models
emotion_model = load_model('emotions_model.h5')
age_gender_model = load_model('age_gender_model.h5')

# Label mappings
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
gender_map = {0: 'Male', 1: 'Female'}
age_range_map = {
    0: "0-2", 1: "3-9", 2: "10-19", 3: "20-29",
    4: "30-39", 5: "40-49", 6: "50-59", 7: "60+"
}

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_rgb = rgb[y:y+h, x:x+w]

        # Emotion Prediction (48x48 Grayscale)
        emotion_input = cv2.resize(roi_gray, (48, 48))
        emotion_input = emotion_input.reshape(1, 48, 48, 1).astype('float32') / 255.0
        emotion_pred = emotion_model.predict(emotion_input, verbose=0)
        emotion = emotion_labels[np.argmax(emotion_pred)]

        # Age & Gender Prediction (96x96 RGB)
        try:
            age_gender_input = cv2.resize(roi_rgb, (96, 96))
            age_gender_input = img_to_array(age_gender_input)
            age_gender_input = preprocess_input(age_gender_input)
            age_gender_input = np.expand_dims(age_gender_input, axis=0)

            gender_pred, age_pred = age_gender_model.predict(age_gender_input, verbose=0)

            # Handle gender prediction: sigmoid (1 node) or softmax (2 nodes)
            if gender_pred.shape[-1] == 1:
                gender = gender_map[int(gender_pred[0][0] > 0.5)]
            else:
                gender = gender_map[np.argmax(gender_pred)]

            age = age_range_map[np.argmax(age_pred)]

        except Exception as e:
            gender = "Unknown"
            age = "Unknown"

        # Display on frame
        label = f"{emotion}, {gender}, {age}"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 200, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 255, 0), 2)

    cv2.imshow("Real-time Emotion, Gender & Age Prediction", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
